In [1]:
import httpx
from pysolr import Solr

CREATE CORE

In [2]:
headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'create': {
        'name': 'authorities',
        'configSet': '_default',
    },
}

response = httpx.post(
    'http://localhost:8983/api/cores', 
    headers=headers, 
    json=json_data, 
    timeout=None)
response

<Response [200 OK]>

In [3]:
headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'create': {
        'name': 'acervo',
        'configSet': '_default',
    },
}

response = httpx.post(
    'http://localhost:8983/api/cores', 
    headers=headers, 
    json=json_data, 
    timeout=None)
response

<Response [200 OK]>

Authorities Schema

In [7]:
url = "http://localhost:8983/api/cores/authorities/schema"

headers = {
    'Content-type': 'application/json',
}

CREATE FIELD

In [8]:
headers = {
    'Content-type': 'application/json',
}

json_data = {
    'add-field': {
        'name': 'generationDate',
        'type': 'text_general',
        'stored': True,
    },
}

response = httpx.post(
    'http://localhost:8983/api/cores/authorities/schema', 
    headers=headers, 
    json=json_data,
    timeout=None)

response

<Response [200 OK]>

EDIT FIELD

In [3]:
json_data = {
  "replace-field":{
     "name":"deathDate",
     "type":"text_general"
    }
}

headers = {
    'Content-type': 'application/json',
}

response = httpx.post(
    'http://localhost:8983/api/cores/authorities/schema', 
    headers=headers, 
    json=json_data,
    timeout=None)

response

<Response [200 OK]>

Add a New Copy Field Rule

In [6]:
json_data = {
    'add-copy-field': {
        'source': 'fullerName',
        'dest': [
            'general_search',
        ],
    },
}

response = httpx.post(
    url, 
    headers=headers, 
    json=json_data,
    timeout=None)

response

<Response [200]>

In [8]:
json_data = {
    'add-copy-field': {
        'source': 'name',
        'dest': [
            'general_search',
        ],
    },
}

response = httpx.post(
    url, 
    headers=headers, 
    json=json_data,
    timeout=None)

response

<Response [200 OK]>

In [9]:
json_data = {
    'add-copy-field': {
        'source': 'variants',
        'dest': [
            'general_search',
        ],
    },
}

response = httpx.post(
    url, 
    headers=headers, 
    json=json_data,
    timeout=None)

response

<Response [200 OK]>

In [12]:
url = "http://localhost:8983/solr/admin/cores?action=REINDEXCOLLECTION&name=authorities"
r = httpx.post(url)
r

json_data = {
    'action': {
        'name': 'authorities',
        'configSet': '_default',
    },
}
r = "http://localhost:8983/solr/admin/collections?action=REINDEXCOLLECTION&name=techproducts_v2"


<Response [400 Bad Request]>

PYSOLR

In [3]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":17,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-2"}},\n  "status":"OK"}\n'

In [3]:
r = solr.search(q="*:*")

DELETE ALL

In [10]:
solr.delete(q='*:*', commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">37</int>\n</lst>\n</response>\n'

ADD DOCS

Work

In [5]:
work = {
        "id": "bk-1",
        "type": "work",
        "content": "text",
        "mainTitle": "Os donos da paisagem: estudos",
        "language": "Portguês",
        "cdd": "352.6",
        "cutter": "O48d",
        "subjects": [{"id": "bk-2",
                      "type": "topic", 
                      "authority": "Geografia",
                      "subjectOf": ["bk-1"]
                      }]
    }

solr.add([work], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":131}}\n'

subjects

In [6]:
subjects = {"id": "bk-3",
            "type": "topic", 
            "authority": "Paisagem" }
solr.add([subjects], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":85}}\n'

Adiciona subjects ao work

In [7]:
up = {
  "id": "bk-1",
  "subjects": { "add": { "id": "bk-3"} }
}

solr.add([up], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":112}}\n'

Indexing Nested Documents

In [5]:
doc = [{ "id": "bk-test",
   "mainTitle": "Swingline Stapler",
   "subjects": [ { "id": "bk-subject1",
               "term_topic": "RED"
             },
             { "id": "bk-subject2",
               "term_topic": "BLACK"
             } ]
 }]
solr.add(doc, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":445}}\n'

In [6]:
doc =  {"id":"bk-2",
        "bibrame":"work",
        "contentType":"Texto",
        "mainTitle":"Conjecturas e refutações",
        "language":"Português",
        "cdd":"548.2",
        "cutter":"A587s",
        "subjects":[{
    "id":"sh85084414",
        "type":"Topic",
        "authority":"Metodologia",
        "variants":["Filosofia--Metodologia"],
        "narrowers":["Solução de problemas",
          "Genealogia (Filosofia)",
          "Formalização (Filosofia)",
          "experimentos mentais",
          "Análise (Filosofia)",
          "Performativo (Filosofia)",
          "Classificação das ciências",
          "Abordagem interdisciplinar do conhecimento",
          "Heurística",
          "método longitudinal"],
        "reciprocalAuthoritys":["Pesquisa"]
        }]}

solr.add([doc], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":242}}\n'

In [7]:
up = {
  "id": "bk-2",
  "subjects": { "add": { "id": "subject-TEST",
                        "authority": "Staplers"
                      } }
}

solr.add([up], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":681}}\n'

DSPACE 7 SOLR

In [8]:
ri = Solr('http://45.35.215.157:8983/solr/search', timeout=10)
ri.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":2,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"search_text",\n      "q.op":"AND",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"-17076"}},\n  "status":"OK"}\n'

In [9]:
r = ri.search("*:*")
r